In [23]:
import numpy as np

# Needs the lower triangular code

def lower_triangular_solve(A, b):

    # we should take care to ensure that arrays are stored with the correct type - float!
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape} {b.shape}')
    
    # checks whether A is lower triangular
    for i in range(n):
        for j in range(i+1,n):
            if not np.isclose(A[i, j], 0.0):
                raise ValueError(f'A is not lower triangular! {A[i, j]}')

    # checks whether A has zero diagonal element
    for i in range(n):
        if np.isclose(A[i, i], 0.0):
            raise ValueError(f'A[{i}, {i}] is zero')
    
    # create a new array to store the results
    x = np.empty_like(b)
    
    # perform forward substitution
    x[0] = b[0] / A[0, 0]
    for i in range(1,n):
        x[i] = b[i] / A[i, i]
        for j in range(i):
            x[i] = x[i] - A[i,j]*x[j]/A[i, i]
        
    return x


def Gauss_Seidel_iteration(A, b, max_iteration, x0 = None):
    # we should take care to ensure that arrays are stored with the correct type - float!
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape} {b.shape}')

    for i in range(n):
        if np.isclose(A[i, i], 0):
            raise ValueError(f'A[{i}, {i}] is zero')

    # do not construct iteration matrices explicitly
    LD = np.zeros_like(A)
    U = np.zeros_like(A)
    for i in range(n):
        for j in range(n):
            if i < j:
                U[i, j] = A[i, j]
            else:
                LD[i, j] = A[i, j]
    
    # p = (L + D)^{-1} b --> found by solving triangular system
    # (L + D) p = b
    p = lower_triangular_solve(LD, b)
      
    #create a new array to store the results, initialised as zero
    if x0 is None:
        x = np.zeros_like(b)
    else:
        x = x0.copy()
        
    # perform iteration x <- p - P * x
    # (L+D)(xnew - p) = U*x
    Ux = np.empty_like(x)
    for it in range(max_iteration):
        for i in range(n):
            Ux[i] = 0.0
            for j in range(i+1, n):
                Ux[i] += U[i, j] * x[j]
        Px = lower_triangular_solve(LD, Ux)
        x = p - Px
                
    return x

A = np.array([[2, -1, 0], [-1, 1, 3], [1, 0, 1]])
b = np.array([1, 3, 2])
# Testing using this 
# numpy linear solver
aM = np.array([0,0,0])
#x0 = np.linalg.solve(A,b)
#print("Solution by numpy solver:", x0)
x = Gauss_Seidel_iteration(A, b, 1, aM).astype(np.float64)
print("Solution by Gauss Seidel iteration: ", x)
print("Residual: ", np.matmul(A,x)-b)
x = np.array([0,0,0])
print("Residual for initial: ", np.matmul(A,x)-b)

Solution by Gauss Seidel iteration:  [0.5 3.5 1.5]
Residual:  [-3.5  4.5  0. ]
Residual for initial:  [-1 -3 -2]


In [21]:
import numpy as np

def gauss_seidel(A, b, x0, tol=1e-6, max_iter=100):
    n = A.shape[0]
    x = x0
    for i in range(max_iter):
        x_new = np.zeros(n)
        for j in range(n):
            s1 = np.dot(A[j, :j], x_new[:j])
            s2 = np.dot(A[j, j+1:], x[j+1:])
            x_new[j] = (b[j] - s1 - s2)/A[j, j]
        if np.allclose(x, x_new, rtol=tol):
            return x_new
        x = x_new
    return x
    raise ValueError("Gauss-Seidel iteration did not converge")

    
A = np.array([[2, -1, 0], [-1, 1, 3], [1, 0, 1]])
b = np.array([1, 3, 2])
# Testing using this 
# numpy linear solver
aM = np.array([0,0,0])
#x0 = np.linalg.solve(A,b)
#print("Solution by numpy solver:", x0)
x = gauss_seidel(A, b, aM, 1e-6, 1)
print("Solution by Gauss Seidel iteration: ", x)
print("Residual: ", np.matmul(A,x)-b)

Solution by Gauss Seidel iteration:  [0.5 3.5 1.5]
Residual:  [-3.5  4.5  0. ]
